# Parquet of the base dataframe

A Parquet file is a columnar storage file format optimized for use with big data processing frameworks, which efficiently compresses and encodes the data in a DataFrame for faster read and write operations.

To transform into parquet we use :
- pandas
- pyarrow (as engine)

Categorical => LabelBinarizer or OneHotEncoder or Label Encoder (one dimension with different values (1, 2, 3, 4, etc.))
- SEASON (4)
- BASIN (7)
- NATURE (6)

Numeric => everything between 0 and 1
- LAT
- LON
- WIND 
- DIST2LAND
- STORM_SPEED
- STORM_DIR

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Dataset : from csv to DataFrame
df = pd.read_csv(
    "../data/ibtracs.csv", skiprows=[1], header=0, low_memory=False
)

In [3]:
# working on a copy of the dataset to preserve an untouched source of the dfcon
df_ibtracs = df.copy()

In [4]:
# Prevent downcasting in the .replace
pd.set_option("future.no_silent_downcasting", True)

for col in df_ibtracs.columns:
    df_ibtracs[col] = df_ibtracs[col].replace(" ", np.nan)
    try:
        df_ibtracs[col] = pd.to_numeric(df_ibtracs[col])
    except ValueError:
        pass

In [5]:
df_ibtracs = df_ibtracs.dropna(subset=["TD9636_STAGE"])

In [6]:
df_ibtracs["ISO_TIME"] = pd.to_datetime(df_ibtracs["ISO_TIME"])

In [7]:
def get_season(date, latitude):
    month = date.month
    day = date.day

    if latitude >= 0:  # Northern Hemisphere
        match month:
            case 12 | 1 | 2:
                return "winter"
            case 3 | 4 | 5:
                return "spring"
            case 6 | 7 | 8:
                return "summer"
            case 9 | 10 | 11:
                return "fall"

    else:  # Southern Hemisphere
        match month:
            case 12 | 1 | 2:
                return "summer"
            case 3 | 4 | 5:
                return "fall"
            case 6 | 7 | 8:
                return "winter"
            case 9 | 10 | 11:
                return "spring"

In [8]:
df_ibtracs["SEASON"] = df_ibtracs.apply(
    lambda row: get_season(row["ISO_TIME"], row["LAT"]), axis=1
)

In [9]:
# renaming USA_WIND as WIND
df_ibtracs.rename(columns={"USA_WIND": "WIND"}, inplace=True)

In [10]:
df_ibtracs[
    ["BOM_WIND", "WELLINGTON_WIND", "REUNION_WIND", "HKO_WIND", "TOKYO_WIND"]
] = (
    df_ibtracs[
        [
            "BOM_WIND",
            "WELLINGTON_WIND",
            "REUNION_WIND",
            "HKO_WIND",
            "TOKYO_WIND",
        ]
    ]
    * 1.12
)

In [11]:
wind_list = [
    "TD9636_WIND",
    "NEUMANN_WIND",
    "HKO_WIND",
    "TOKYO_WIND",
    "BOM_WIND",
    "WELLINGTON_WIND",
    "REUNION_WIND",
    "DS824_WIND",
]

In [12]:
# if WIND is NaN we loop over the other columns to fill it
for column in wind_list:
    df_ibtracs["WIND"] = df_ibtracs["WIND"].fillna(value=df_ibtracs[column])

In [13]:
# Drop rows where WIND is still null
df_ibtracs = df_ibtracs.dropna(subset=["WIND"])

In [14]:
df_ = df_ibtracs[
    [
        "SID",
        "ISO_TIME",
        "SEASON",
        "BASIN",
        "NATURE",
        "LAT",
        "LON",
        "WIND",
        "DIST2LAND",
        "STORM_SPEED",
        "STORM_DIR",
        "TD9636_STAGE",
    ]
]

In [15]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45025 entries, 0 to 67409
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   SID           45025 non-null  object        
 1   ISO_TIME      45025 non-null  datetime64[ns]
 2   SEASON        45025 non-null  object        
 3   BASIN         45025 non-null  object        
 4   NATURE        45025 non-null  object        
 5   LAT           45025 non-null  float64       
 6   LON           45025 non-null  float64       
 7   WIND          45025 non-null  float64       
 8   DIST2LAND     45025 non-null  int64         
 9   STORM_SPEED   45025 non-null  float64       
 10  STORM_DIR     45025 non-null  float64       
 11  TD9636_STAGE  45025 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 4.5+ MB


In [16]:
df_.isnull().sum().sort_values()

SID             0
ISO_TIME        0
SEASON          0
BASIN           0
NATURE          0
LAT             0
LON             0
WIND            0
DIST2LAND       0
STORM_SPEED     0
STORM_DIR       0
TD9636_STAGE    0
dtype: int64

## Isolating data

We need to drop some rows so that the model is not test/trained on them and so that they are not inserted into the small database in the app from the get-go. We are dropping :
- the most recent cyclone 
- the last 2 rows of the second most recent cyclone. 

In [18]:
# Drop the most recent sid (which will be the last in the sorted df)
most_recent_sid = df['SID'].iloc[-1]
dropped_df = df[df['SID'] == most_recent_sid]
df = df[df['SID'] != most_recent_sid]

# Drop the two most recent rows from the remaining data
recent_rows = df.iloc[-2:]
dropped_df = pd.concat([dropped_df, recent_rows])
df = df.iloc[:-2]

In [19]:
dropped_df

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
297063,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 06:00:00,NR,17.8,145.5,...,,,,,,,,,5,355
297064,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 09:00:00,NR,18.1,145.5,...,,,,,,,,,7,0
297065,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 12:00:00,NR,18.5,145.5,...,,,,,,,,,11,5
297066,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 15:00:00,NR,19.2,145.6,...,,,,,,,,,14,10
297067,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 18:00:00,NR,19.9,145.7,...,,,,,,,,,13,10
297068,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-06 21:00:00,NR,20.4,145.8,...,,,,,,,,,10,15
297069,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-07 00:00:00,NR,20.9,146.0,...,,,,,,,,,9,25
297070,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-07 03:00:00,NR,21.3,146.2,...,,,,,,,,,9,30
297071,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-07 06:00:00,NR,21.7,146.5,...,,,,,,,,,13,25
297072,2024280N18146,2024,72,WP,MM,BARIJAT,2024-10-07 09:00:00,NR,22.5,146.9,...,,,,,,,,,16,25


## Storing the dataframe

In [20]:
# From DataFrame to parquet
df_.to_parquet("../data/app_db.parquet", engine="pyarrow")